<a href="https://colab.research.google.com/github/dzurel/Google-QUEST-Q-A-Labeling/blob/master/Google_QUEST_FastAI_BertEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling, selection, and evaluation
from fastai.text import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder

%matplotlib inline 

import tensorflow as tf

import lightgbm as lgb
import xgboost as xgb

In [2]:
!pip install fastai

In [0]:
#INPUT_PATH="/kaggle/input/"
#INPUT_PATH="../input/"
train = pd.read_csv("https://raw.githubusercontent.com/dzurel/Google-QUEST-Q-A-Labeling/master/train.csv")
train = train.reindex(np.random.permutation(train.index))
test = pd.read_csv("https://raw.githubusercontent.com/dzurel/Google-QUEST-Q-A-Labeling/master/test.csv")
submission = pd.read_csv("https://raw.githubusercontent.com/dzurel/Google-QUEST-Q-A-Labeling/master/sample_submission.csv")

In [4]:
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
2770,4414,How to measure a crooked room?,I am faced with the task of measuring a room t...,Björn Pollex,https://diy.stackexchange.com/users/4558,Forget it!!!! Trying to measure an untrue roo...,shirlock homes,https://diy.stackexchange.com/users/386,http://diy.stackexchange.com/questions/10590/h...,LIFE_ARTS,diy.stackexchange.com,1.000000,0.333333,0.0,0.0,1.000000,1.0,1.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,1.000000,1.0,0.000000,0.0,1.000000
2466,3933,Do real and reactive energy exist?,Are there any such things as real and reactive...,Sri,https://electronics.stackexchange.com/users/34625,"Energy is just power integrated over time, so ...",Olin Lathrop,https://electronics.stackexchange.com/users/4512,http://electronics.stackexchange.com/questions...,TECHNOLOGY,electronics.stackexchange.com,1.000000,0.555556,0.0,0.0,1.000000,0.0,0.888889,0.888889,0.666667,0.0,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,1.000000,0.0,0.000000,1.0,1.000000
2074,3304,linq to xml query with all attributes,I have a xml document like this\n\n&lt;Product...,शेखर,https://stackoverflow.com/users/853609,XDocument xdoc = XDocument.Load(Server.MapPath...,Fung,https://stackoverflow.com/users/570036,http://stackoverflow.com/questions/15333674/li...,STACKOVERFLOW,stackoverflow.com,0.888889,0.555556,0.0,0.5,1.000000,1.0,0.333333,0.333333,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.666667,0.0,0.0,0.777778,0.888889,0.555556,1.000000,1.000000,0.866667,1.0,0.666667,0.0,0.888889
5070,8063,pre hung doors to fit into old frame,"I have purchased some pre hung doors at 30"" by...",brian smith,https://diy.stackexchange.com/users/20542,"Take off the trim, remove the old frame, insta...",iLikeDirt,https://diy.stackexchange.com/users/15879,http://diy.stackexchange.com/questions/40199,LIFE_ARTS,diy.stackexchange.com,0.888889,0.333333,0.0,1.0,0.666667,1.0,0.777778,0.555556,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.444444,0.777778,0.555556,0.777778,0.888889,1.000000,1.0,0.000000,0.0,0.888889
2215,3529,Would a meta-answer be a good idea?,One of the stated goals of the site was the ha...,Yishai,https://meta.stackexchange.com/users/130225,I think this would be useful in the case of qu...,intuited,https://meta.stackexchange.com/users/145298,http://meta.stackexchange.com/questions/27568/...,TECHNOLOGY,meta.stackexchange.com,1.000000,1.000000,0.0,1.0,1.000000,1.0,0.666667,0.333333,1.000000,0.0,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.000000,0.666667,1.000000,0.666667,1.000000,0.800000,0.0,0.000000,1.0,1.000000


In [0]:
targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ]

input_columns = ['question_title','question_body','answer']

In [0]:
#clean data
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))

    return df

In [0]:
train = clean_data(train, input_columns)
test = clean_data(test, input_columns)

In [8]:
train.head(4)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
2770,4414,how to measure a crooked room ?,i am faced with the task of measuring a room t...,Björn Pollex,https://diy.stackexchange.com/users/4558,forget it ! ! ! ! trying to measure an un...,shirlock homes,https://diy.stackexchange.com/users/386,http://diy.stackexchange.com/questions/10590/h...,LIFE_ARTS,diy.stackexchange.com,1.000000,0.333333,0.0,0.0,1.000000,1.0,1.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,1.000000,1.0,0.000000,0.0,1.000000
2466,3933,do real and reactive energy exist ?,are there any such things as real and reactive...,Sri,https://electronics.stackexchange.com/users/34625,"energy is just power integrated over time , s...",Olin Lathrop,https://electronics.stackexchange.com/users/4512,http://electronics.stackexchange.com/questions...,TECHNOLOGY,electronics.stackexchange.com,1.000000,0.555556,0.0,0.0,1.000000,0.0,0.888889,0.888889,0.666667,0.0,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,1.000000,0.0,0.000000,1.0,1.000000
2074,3304,linq to xml query with all attributes,i have a xml document like this & lt ; product...,शेखर,https://stackoverflow.com/users/853609,xdocument xdoc = xdocument . load ( server ....,Fung,https://stackoverflow.com/users/570036,http://stackoverflow.com/questions/15333674/li...,STACKOVERFLOW,stackoverflow.com,0.888889,0.555556,0.0,0.5,1.000000,1.0,0.333333,0.333333,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.666667,0.0,0.0,0.777778,0.888889,0.555556,1.000000,1.000000,0.866667,1.0,0.666667,0.0,0.888889
5070,8063,pre hung doors to fit into old frame,i have purchased some pre hung doors at # # ...,brian smith,https://diy.stackexchange.com/users/20542,"take off the trim , remove the old frame , i...",iLikeDirt,https://diy.stackexchange.com/users/15879,http://diy.stackexchange.com/questions/40199,LIFE_ARTS,diy.stackexchange.com,0.888889,0.333333,0.0,1.0,0.666667,1.0,0.777778,0.555556,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.444444,0.777778,0.555556,0.777778,0.888889,1.000000,1.0,0.000000,0.0,0.888889


In [9]:
train['question_body'][0]

'after playing around with macro photography on - the - cheap  ( read :  reversed lens ,  rev .  lens mounted on a straight lens ,  passive extension tubes )  ,  i would like to get further with this .  the problems with the techniques i used is that focus is manual and aperture control is problematic at best .  this limited my setup to still subjects  ( read :  dead insects )  now ,  as spring is approaching ,  i want to be able to shoot live insects .  i believe that for this ,  autofocus and settable aperture will be of great help . so ,  one obvious but expensive option is a macro lens  ( say ,  ef  #  #  # mm macro )  however ,  i am not really interested in yet another prime lens .  an alternative is the electrical extension tubes . except for maximum focusing distance ,  what am i losing when using tubes  ( coupled with a fine lens ,  say ef #  #  -  #  #  #  / 2 . 8 )  instead of a macro lens ? '

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6079 entries, 2770 to 3214
Data columns (total 41 columns):
qa_id                                    6079 non-null int64
question_title                           6079 non-null object
question_body                            6079 non-null object
question_user_name                       6079 non-null object
question_user_page                       6079 non-null object
answer                                   6079 non-null object
answer_user_name                         6079 non-null object
answer_user_page                         6079 non-null object
url                                      6079 non-null object
category                                 6079 non-null object
host                                     6079 non-null object
question_asker_intent_understanding      6079 non-null float64
question_body_critical                   6079 non-null float64
question_conversational                  6079 non-null float64
question_expect_shor

In [0]:
train.sort_index(inplace=True)
test[['question_body']].to_csv('Filtered_test.csv', index=False, encoding='utf-8')
train[['question_body']].to_csv('Filtered_train.csv', index=False, encoding='utf-8')

In [0]:
def clean_ascii(text):
    # function to remove non-ASCII chars from data
    return ''.join(i for i in text if ord(i) < 128)

In [0]:
train['question_body'] = train['question_body'].apply(clean_ascii)

### Tokenization

We read in the data and add new words to our dictionary, as well as create a representation of words using numbers.

In [26]:
data = TextClasDataBunch.from_csv('.', 'Filtered_train.csv')
data.show_batch()

IndexError: ignored